# Setting up Media Pipe

In [1]:
# Importing libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils # Drawing Utilites
mp_pose = mp.solutions.pose # Pose
mp_holistic = mp.solutions.holistic # Holistic

In [2]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False # Image is no longer writeable
    results = model.process(image) # Make prediction
    image.flags.writeable = True # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# Real Time Splits Detection

In [13]:
class Rating:
    def __init__(self,a,b,c):
        self.a = np.array(a)
        self.b = np.array(b)
        self.c = np.array(c)
        self.radians = np.arctan2(self.c[1]-self.b[1], self.c[0]-self.b[0]) - np.arctan2(self.a[1]-self.b[1], self.a[0]-self.b[0])
        self.angle = np.abs(self.radians*180.0/np.pi)
        
    def calculate_angle(self):
        if self.angle > 180.0:
            self.angle = 360-self.angle
            
        return self.angle
    
    def calculate_rating_splits(self):
        if self.angle >= 180:
            percentage = 100 * (180/self.angle)
        else:
            self.angle = 360 - self.angle
            percentage = 100 * (180/self.angle)
        
        return percentage
    
    def calculate_rating_splits_back(self):
        if self.angle >= 90:
            percentage = 100 * (90/self.angle)
        else:
            self.angle = 180 - self.angle
            percentage = 100 * (90/self.angle)
        
        return percentage

In [14]:
def draw_styled_landmarks_splits(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, [list(mp_holistic.POSE_CONNECTIONS)[7], list(mp_holistic.POSE_CONNECTIONS)[29], list(mp_holistic.POSE_CONNECTIONS)[13], list(mp_holistic.POSE_CONNECTIONS)[10], list(mp_holistic.POSE_CONNECTIONS)[5], list(mp_holistic.POSE_CONNECTIONS)[27], list(mp_holistic.POSE_CONNECTIONS)[33]], 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=3, circle_radius=3)
                             )

In [17]:
# Splits

cap = cv2.VideoCapture(0)

# Access mediapipe model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
# Set mediapipe model 
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make Detections
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Draw landmarks
        draw_styled_landmarks_splits(image, results)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            
            # Calculate percentage
            obj_right = Rating(right_knee,right_hip,left_hip)
            right_percentage = obj_right.calculate_rating_splits()
            
            obj_left = Rating(left_knee,left_hip,right_hip)
            left_percentage = obj_left.calculate_rating_splits()
            
            percentage = (right_percentage + left_percentage)/2
            
            obj = Rating(right_knee,right_hip,right_shoulder)
            back_angle = obj.calculate_angle()
            back_percentage = obj.calculate_rating_splits_back()
            
            # Display Green or Red Message
            if percentage > 80 and back_percentage > 80:
                cv2.putText(image, "Perfect!", 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 128, 0), 2, cv2.LINE_AA
                                )
            elif back_percentage < 80 and back_angle > 90:
                cv2.putText(image, "Move your back forward", 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                               )
            elif back_percentage < 80 and back_angle < 90:
                cv2.putText(image, "Move your back backward", 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                               )
            else:
                cv2.putText(image, "Lower your legs", 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                                )
                       
            # Rectangle Setup on Top
            cv2.rectangle(image, (0,0), (10000,80), (245,117,16), -1)
            
            # Display Leg Accuarcy
            cv2.putText(image, 'Split Accuarcy', (300,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(percentage, 2)) + '%', 
                        (300,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,127,250), 2, cv2.LINE_AA)
                    
            # Display Back Accuarcy
            cv2.putText(image, 'Back Accuarcy', (800,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(back_percentage, 2)) + '%', 
                        (800,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,127,250), 2, cv2.LINE_AA)
        except:
            pass
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()